In [197]:
import pandas as pd

In [198]:
df = pd.read_parquet('../data/transformed/movies_overview.parquet')

In [199]:
df['overview'].values

array(['Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.',
       'The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.',
       'Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.',
       ...,
       'We are Global Online Quran Teachers, an educational organization that i

In [200]:
sample = df.sample(n=1000)['overview']
sample

370179     The Amish: Barn Raising Day examines how Amish...
371097     The hottest, horniest hunks are found under th...
983192     "Pandamonium" is an ethnographic essay film wh...
179673     Chanell Heart and Osa Lovely - Step mom and da...
1016114    A man looks for his missing dog in the middle ...
                                 ...                        
1099922    A documentary film project that follows two tr...
680786     This documentary explores the question of why ...
921530     This award-winning film documents the only upr...
583452     A surreal short film that uses dreamlike image...
653358                                                      
Name: overview, Length: 1000, dtype: object

In [201]:
sample = df.sample(n=10000)

In [202]:
sample = sample[~sample['overview'].isnull()]
sample['ascii'] = sample['overview'].apply(lambda row: row.isascii())
sample = sample[sample['ascii'] == True]

In [203]:
from sklearn.feature_extraction.text import CountVectorizer

ids = sample['id'].values  # Extract IDs
documents = sample['overview'].values
# Initialize the CountVectorizer
vectorizer = CountVectorizer(lowercase=True, stop_words='english')

bow_matrix = vectorizer.fit_transform(documents)
# Fit and transform the documents into the BoW model

bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
bow_df.insert(0, 'id', ids)  # Insert ID at the beginning
bow_df



ValueError: cannot insert id, already exists

In [ ]:
import pandas as pd

def filter(df: pd.DataFrame):
    df.drop(columns=df.columns[df.sum() < 5], inplace=True)

filter(bow_df)

In [ ]:
metadata = pd.read_parquet('../data/transformed/movies_metadata.parquet')
genres = metadata[['id','genre_1']]
del metadata

In [ ]:
df = bow_df.merge(genres, on="id")
df

,id,000,10,100,11,12,13,14,15,16,...,young,younger,youth,zealand,zero,zombie,zombies,zone,zoo,genre_1
0,279592,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama
1,27824,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
2,109785,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
3,973929,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Horror
4,132020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,1203613,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
4128,459833,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,Documentary
4129,873580,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Music
4130,1314142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Animation


In [ ]:
df = df[~df['genre_1'].isnull()]

In [ ]:
len(df[df['genre_1'] == df['genre_1'].mode().iloc[0]])

502

In [ ]:
    df = df[(df['genre_1'] == 'Horror') | (df['genre_1'] == 'Family')]

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Splitting features and target
X = df.drop(columns=['genre_1', 'id'])  # Features (word counts)
y = df['genre_1']  # Target (genre)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95
